<a href="https://colab.research.google.com/github/Avastbinder/CSE450/blob/main/notebooks/starter_bikes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install lets_plot
!pip install tensorflow

In [39]:
import pandas as pd
from lets_plot import *
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

bikes = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')

In [40]:
bikes["users"] = bikes["casual"] + bikes["registered"]
bikes.drop(columns=["casual", "registered"], inplace=True)
bikes['month'] = pd.DatetimeIndex(bikes['dteday']).month
bikes['year'] = pd.DatetimeIndex(bikes['dteday']).year
bikes.drop(columns=["dteday"], inplace=True)
features = ['month', 'year', 'season', 'hr', 'holiday', 'workingday', 'weathersit', 'temp_c', 'feels_like_c', 'hum', 'windspeed']

In [41]:
X = pd.get_dummies(bikes[features], drop_first=True)
y = bikes["users"]

In [42]:
train_dataset = bikes.sample(frac=0.8, random_state=0)
test_dataset = bikes.drop(train_dataset.index)

train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('users')
test_labels = test_features.pop('users')


In [44]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))
first = np.array(train_features[:1])

with np.printoptions(precision=2, suppress=True):
  print('First example:', first)
  print()
  print('Normalized:', normalizer(first).numpy())

First example: [[  13.      8.4     7.9     0.63    5.1     1.      1.      0.      0.
     2.   2020.  ]]

Normalized: [[ 0.22 -0.71 -0.59 -0.04 -1.02 -0.59 -1.36 -0.18 -1.47 -1.3   0.83]]


In [ ]:
linear_model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=1)
])
linear_model.predict(train_features[:10])
linear_model.layers[1].kernel
linear_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

history = linear_model.fit(
    train_features, train_labels,
    epochs=100,
    # suppress logging
    verbose=0,
    # Calculate validation results on 20% of the training data
    validation_split = 0.2)

test_results = {}
test_results['linear_model'] = linear_model.evaluate(
    test_features, test_labels, verbose=0)
ggplot(history) + geom_line()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step


In [50]:
def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model